In [1]:
%load_ext autoreload
%autoreload 2

import sys
import tqdm
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt

from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning import Trainer
from transformers import T5Tokenizer, T5EncoderModel
from sklearn.metrics import classification_report

sys.path.append('..')
from pLMtrainer.dataloader import FrustrationDataset, FrustrationDataModule
from pLMtrainer.models import FrustrationFNN

torch.set_float32_matmul_precision('medium')

/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current working directory: /Users/janleusch/Documents/phd/pLMtrainer/pLMtrainer/notebooks


In [10]:
a = torch.Tensor([[[1,2,3],
                   [4,5,6],
                   [0,0,0]],
                   
                   [[7,8,9],
                    [10,11,12],
                    [0,0,0]],
                    
                    [[13,14,15], 
                     [16,17,18],
                     [0,0,0]]])
a

tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 0.,  0.,  0.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.],
         [ 0.,  0.,  0.]],

        [[13., 14., 15.],
         [16., 17., 18.],
         [ 0.,  0.,  0.]]])

In [31]:
b = torch.zeros(size=(3,10,22))
b.shape

torch.Size([3, 10, 22])

In [32]:
b[..., -1]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [40]:
c = torch.ones(size=(3,10), dtype=torch.bool)
c.flatten().shape

torch.Size([30])

In [42]:
b[:,:, :-1].flatten(0,1).shape

torch.Size([30, 21])

In [2]:
parquet_path = "../data/frustration/v3_frustration.parquet.gzip"

In [30]:
data_module = FrustrationDataModule(parquet_path=parquet_path, regression=False, batch_size=10, max_seq_length=100, num_workers=1, persistent_workers=True)

In [31]:
model = FrustrationFNN.load_from_checkpoint(checkpoint_path="../data/cluster_runs/debug-v4.ckpt",
                                            input_dim=1024, 
                                            hidden_dim=32, 
                                            output_dim=21, 
                                            dropout=0.15, 
                                            max_seq_length=100,
                                            regression=False, 
                                            pLM_model="../data/ProstT5", 
                                            pLM_precision="full", 
                                            prefix_prostT5="<AA2fold>")

In [32]:
logger = CSVLogger("./checkpoints", name="pred_logs")
trainer = Trainer(accelerator='auto',
                  devices=-1, 
                  max_epochs=5,
                  logger=logger,
                  log_every_n_steps=10, 
                  precision="16-mixed",
                  gradient_clip_val=1,
                  enable_progress_bar=True,
                  )

Using 16bit Automatic Mixed Precision (AMP)
/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [33]:
trainer.test(model, datamodule=data_module)

Loaded 1000 samples from ../data/frustration/v3_frustration.parquet.gzip
Created train/val/test masks
Initialized res_idx_mask and frst_vals tensors
Populated res_idx_mask and frst_vals tensors
Created train dataset
Created val dataset
Created test dataset
Train/Val/Test split: 899/35/66 samples


/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
python(11325) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Testing DataLoader 0:   0%|          | 0/7 [00:00<?, ?it/s]

/Users/janleusch/anaconda3/envs/biotrainer/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Testing DataLoader 0: 100%|██████████| 7/7 [00:08<00:00,  0.85it/s]Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00        55
           7       0.42      0.31      0.36       269
           8       0.55      0.66      0.60       465
           9       0.36      0.32      0.34       256
          10       0.32      0.19      0.24       289
          11       0.31      0.37      0.34       386
          12       0.37      0.50      0.43       500
          13       0.33      0.39      0.36       258
          14       0.00      0.00      0.00        78
          15       0.00      0.00      0.00        29
          16       0.00      0.00      0.00 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.5665206909179688     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.5665206909179688}]

In [34]:
model.save_preds_targets("frustration_preds_targets.npz")

In [44]:
report = classification_report(model.all_targets, model.all_preds, labels=range(1, 21), digits=4, zero_division=0)

In [45]:
print(report)

              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000         0
           2     0.0000    0.0000    0.0000         0
           3     0.0000    0.0000    0.0000         3
           4     0.0000    0.0000    0.0000         4
           5     0.0000    0.0000    0.0000        11
           6     0.0000    0.0000    0.0000        55
           7     0.4158    0.3123    0.3567       269
           8     0.5510    0.6624    0.6016       465
           9     0.3649    0.3164    0.3389       256
          10     0.3218    0.1938    0.2419       289
          11     0.3115    0.3705    0.3385       386
          12     0.3690    0.5040    0.4260       500
          13     0.3258    0.3915    0.3556       258
          14     0.0000    0.0000    0.0000        78
          15     0.0000    0.0000    0.0000        29
          16     0.0000    0.0000    0.0000        13
          17     0.0000    0.0000    0.0000         4
          18     0.0000    

In [37]:
df = pq.read_table(parquet_path).to_pandas()

In [39]:
df_test = df[df['set'] == 'test'].copy()

In [46]:
data = pq.ParquetFile(parquet_path)
data

In [48]:
data.num_row_groups

1

In [ ]:
data.read_row_group(1).to_pandas()

In [13]:
all_preds, all_targets, masked_all_preds, masked_all_targets = np.load("frustration_preds_targets.npz").values()

In [25]:
report = classification_report(masked_all_targets, masked_all_preds, labels=range(1, 21), digits=6, zero_division=0)

In [26]:
print(report)

              precision    recall  f1-score   support

           1   0.000000  0.000000  0.000000       810
           2   0.000000  0.000000  0.000000      1003
           3   0.000000  0.000000  0.000000      2546
           4   0.058824  0.000367  0.000729      8183
           5   0.046332  0.000577  0.001140     41605
           6   0.225636  0.002955  0.005833    216259
           7   0.488101  0.399606  0.439443   1159955
           8   0.501561  0.620410  0.554691   1711433
           9   0.355924  0.302541  0.327068   1092732
          10   0.272654  0.170627  0.209899   1159851
          11   0.370899  0.415548  0.391956   1677849
          12   0.361101  0.486381  0.414481   1796416
          13   0.373444  0.458945  0.411804   1110193
          14   0.180708  0.012993  0.024242    310869
          15   0.074534  0.000160  0.000320     74789
          16   0.155427  0.095606  0.118389     33774
          17   0.154472  0.001958  0.003867     19407
          18   0.000000  0.

In [22]:
np.random.seed(42)
random_masked_targets = np.random.permutation(masked_all_targets)

In [27]:
rnd_report = classification_report(masked_all_targets, random_masked_targets, labels=range(1, 21), digits=6, zero_division=0)

In [28]:
print(rnd_report)

              precision    recall  f1-score   support

           1   0.000000  0.000000  0.000000       810
           2   0.000997  0.000997  0.000997      1003
           3   0.000000  0.000000  0.000000      2546
           4   0.000122  0.000122  0.000122      8183
           5   0.003966  0.003966  0.003966     41605
           6   0.020670  0.020670  0.020670    216259
           7   0.111091  0.111091  0.111091   1159955
           8   0.163975  0.163975  0.163975   1711433
           9   0.104585  0.104585  0.104585   1092732
          10   0.111206  0.111206  0.111206   1159851
          11   0.160548  0.160548  0.160548   1677849
          12   0.172167  0.172167  0.172167   1796416
          13   0.106358  0.106358  0.106358   1110193
          14   0.030009  0.030009  0.030009    310869
          15   0.007421  0.007421  0.007421     74789
          16   0.003020  0.003020  0.003020     33774
          17   0.001649  0.001649  0.001649     19407
          18   0.000952  0.